In [1]:
import os
import glob
import time
from datetime import datetime

import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal
from torch.distributions import Categorical

import numpy as np
import pandas as pd

import gym
from machine import Machine
from GymMachEnv import MachineEnv
import matplotlib.pyplot as plt

In [2]:
def gen_param_array(l16_array):
    full = []
    for row in range(len(l16_array)):
        x = l16_array[row]
        arr_params = []
        
        for col in range(len(x)):
            list_params = list(params.values())[col]
            value = list_params[x[col]]
            arr_params.append(value)
        print(arr_params)
        full.append(arr_params)
    return full

def train(params_array,folder_name):
    trial = 0
    for sets in params_array:
        print("trial ",trial)
#         if trial < 6: 
#             trial+=1
#             continue
        evaluate_and_test(folder_name,trial,sets)
        trial+=1
        
def compute_avg_return(environment, policy, num_episodes):
    total_return = 0.0
    for _ in range(num_episodes):
        state = torch.Tensor(environment.reset())
        state = state.unsqueeze(0)
        episode_return = 0.0 
        while not environment.done:
            action = policy.select_action(state,True)
            next_state, reward, done, _ = environment.step(action)
            next_state = torch.Tensor(next_state)
            next_state = next_state.unsqueeze(0)
            state = next_state
            episode_return += reward
        total_return += episode_return   
    avg_return = total_return / num_episodes
    return avg_return# Evaluate the agent's policy once before training.

In [3]:
params = { 'eps_clipping' : [2e-05,2e-04,2e-03,2e-02],
            'gamma' : [0.8,0.9,0.95,0.99],
            'actor_lr' : [1e-04,5e-03,1e-03,1e-02],
            'critic_lr' : [1e-04,5e-03,1e-03,1e-02]}


In [4]:
class RolloutBuffer:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
    

    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]


class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, has_continuous_action_space, action_std_init):
        super(ActorCritic, self).__init__()

        self.has_continuous_action_space = has_continuous_action_space


        self.actor = nn.Sequential(
                            nn.Linear(state_dim,128),
                            nn.ReLU(),
                            nn.Linear(128,64),
                            nn.ReLU(),
                            nn.Linear(64,action_dim),
                            
                            nn.Softmax(dim=-1)
                        )

        
        # critic
        self.critic = nn.Sequential(
                            nn.Linear(state_dim,128),
                            nn.ReLU(),
                            nn.Linear(128,64),
                            nn.ReLU(),
                            nn.Linear(64,1)
                    )
        
    def set_action_std(self, new_action_std):

        if self.has_continuous_action_space:
            self.action_var = torch.full((self.action_dim,), new_action_std * new_action_std)
        else:
            print("--------------------------------------------------------------------------------------------")
            print("WARNING : Calling ActorCritic::set_action_std() on discrete action space policy")
            print("--------------------------------------------------------------------------------------------")


    def forward(self):
        raise NotImplementedError
    

    def act(self, state):

        if self.has_continuous_action_space:
            action_mean = self.actor(state)
            cov_mat = torch.diag(self.action_var).unsqueeze(dim=0)
            dist = MultivariateNormal(action_mean, cov_mat)
        else:
            action_probs = self.actor(state)
            dist = Categorical(action_probs)

        action = dist.sample()
        action_logprob = dist.log_prob(action)
        
        return action.detach(), action_logprob.detach()
    

    def evaluate(self, state, action):

        if self.has_continuous_action_space:
            action_mean = self.actor(state)
            action_var = self.action_var.expand_as(action_mean)
            cov_mat = torch.diag_embed(action_var)
            dist = MultivariateNormal(action_mean, cov_mat)
            
            # for single action continuous environments
            if self.action_dim == 1:
                action = action.reshape(-1, self.action_dim)

        else:
            action_probs = self.actor(state)
            dist = Categorical(action_probs)

        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        state_values = self.critic(state)
        
        return action_logprobs, state_values, dist_entropy


class PPO:
    def __init__(self, state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, has_continuous_action_space, action_std_init=0.6):

        self.has_continuous_action_space = has_continuous_action_space

        if has_continuous_action_space:
            self.action_std = action_std_init

        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.buffer = RolloutBuffer()

        self.policy = ActorCritic(state_dim, action_dim, has_continuous_action_space, action_std_init)
        self.optimizer = torch.optim.Adam([
                        {'params': self.policy.actor.parameters(), 'lr': lr_actor},
                        {'params': self.policy.critic.parameters(), 'lr': lr_critic}
                    ])

        self.policy_old = ActorCritic(state_dim, action_dim, has_continuous_action_space, action_std_init)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.MseLoss = nn.MSELoss()


    def set_action_std(self, new_action_std):
        
        if self.has_continuous_action_space:
            self.action_std = new_action_std
            self.policy.set_action_std(new_action_std)
            self.policy_old.set_action_std(new_action_std)
        
        else:
            print("--------------------------------------------------------------------------------------------")
            print("WARNING : Calling PPO::set_action_std() on discrete action space policy")
            print("--------------------------------------------------------------------------------------------")


    def decay_action_std(self, action_std_decay_rate, min_action_std):
        print("--------------------------------------------------------------------------------------------")

        if self.has_continuous_action_space:
            self.action_std = self.action_std - action_std_decay_rate
            self.action_std = round(self.action_std, 4)
            if (self.action_std <= min_action_std):
                self.action_std = min_action_std
                print("setting actor output action_std to min_action_std : ", self.action_std)
            else:
                print("setting actor output action_std to : ", self.action_std)
            self.set_action_std(self.action_std)

        else:
            print("WARNING : Calling PPO::decay_action_std() on discrete action space policy")

        print("--------------------------------------------------------------------------------------------")


    def select_action(self, state, val=False):

        if val:
            state = torch.FloatTensor(state)
            action, action_logprob = self.policy_old.act(state)
            return action.item()

        else:
            with torch.no_grad():
                state = torch.FloatTensor(state)
                action, action_logprob = self.policy_old.act(state)
            
            self.buffer.states.append(state)
            self.buffer.actions.append(action)
            self.buffer.logprobs.append(action_logprob)

            return action.item()


    def update(self):

        # Monte Carlo estimate of returns
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
            
        # Normalizing the rewards
        rewards = torch.tensor(rewards, dtype=torch.float32)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-7)

        # convert list to tensor
        old_states = torch.squeeze(torch.stack(self.buffer.states, dim=0)).detach()
        old_actions = torch.squeeze(torch.stack(self.buffer.actions, dim=0)).detach()
        old_logprobs = torch.squeeze(torch.stack(self.buffer.logprobs, dim=0)).detach()

        
        # Optimize policy for K epochs
        for _ in range(self.K_epochs):

            # Evaluating old actions and values
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)

            # match state_values tensor dimensions with rewards tensor
            state_values = torch.squeeze(state_values)
            
            # Finding the ratio (pi_theta / pi_theta__old)
            ratios = torch.exp(logprobs - old_logprobs.detach())

            # Finding Surrogate Loss
            advantages = rewards - state_values.detach()   
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages

            # final loss of clipped objective PPO
            loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            
            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
            
        # Copy new weights into old policy
        self.policy_old.load_state_dict(self.policy.state_dict())

        # clear buffer
        self.buffer.clear()
    
    

In [5]:
l16_array = np.array(pd.read_csv('L16.txt',header=None, sep='\t'))
params_array = gen_param_array(l16_array)
params_array

[2e-05, 0.8, 0.0001, 0.0001]
[2e-05, 0.9, 0.005, 0.005]
[2e-05, 0.95, 0.001, 0.001]
[2e-05, 0.99, 0.01, 0.01]
[0.0002, 0.8, 0.005, 0.001]
[0.0002, 0.9, 0.0001, 0.01]
[0.0002, 0.95, 0.01, 0.0001]
[0.0002, 0.99, 0.001, 0.005]
[0.002, 0.8, 0.001, 0.01]
[0.002, 0.9, 0.01, 0.001]
[0.002, 0.95, 0.0001, 0.005]
[0.002, 0.99, 0.005, 0.0001]
[0.02, 0.8, 0.01, 0.005]
[0.02, 0.9, 0.001, 0.0001]
[0.02, 0.95, 0.005, 0.01]
[0.02, 0.99, 0.0001, 0.001]


[[2e-05, 0.8, 0.0001, 0.0001],
 [2e-05, 0.9, 0.005, 0.005],
 [2e-05, 0.95, 0.001, 0.001],
 [2e-05, 0.99, 0.01, 0.01],
 [0.0002, 0.8, 0.005, 0.001],
 [0.0002, 0.9, 0.0001, 0.01],
 [0.0002, 0.95, 0.01, 0.0001],
 [0.0002, 0.99, 0.001, 0.005],
 [0.002, 0.8, 0.001, 0.01],
 [0.002, 0.9, 0.01, 0.001],
 [0.002, 0.95, 0.0001, 0.005],
 [0.002, 0.99, 0.005, 0.0001],
 [0.02, 0.8, 0.01, 0.005],
 [0.02, 0.9, 0.001, 0.0001],
 [0.02, 0.95, 0.005, 0.01],
 [0.02, 0.99, 0.0001, 0.001]]

In [5]:
def running_score_cal(arr,window=50):
    array = np.array(arr)
    df = pd.DataFrame(array[:,1])
    run_score = df.rolling(50,min_periods=1).mean().values.flatten().tolist()
    return run_score[-1]

In [6]:
def evaluate_and_test(folder_name,trial_number,params):
    max_episodes = 40000
    max_ep_len = 100
    update_timestep = max_ep_len * 4      # update policy every n timesteps
    

    machine = Machine()
    machine.curr_state = 0
    env = MachineEnv(machine)

    #Eval Env
    machine2 = Machine()
    machine2.curr_state = 0
    env2 = MachineEnv(machine2)


    gamma = params[1]
    eps_clip = params[0]
    lr_actor = params[2]
    lr_critic = params[3]
    K_epochs = 40
    
    max_score = 0

    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n

    ppo_agent = PPO(state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, False, None)

    # printing and logging variables
    print_running_reward = 0
    print_running_episodes = 0

    log_running_reward = 0
    log_running_episodes = 0

    time_step = 0
    i_episode = 0

    tracker = []
    eval_tracker = []

    print_freq_ep = 10


    # training loop
    while i_episode <= max_episodes:

        state = env.reset()
        current_ep_reward = 0

        for t in range(1, max_ep_len+1):

            # select action with policy
            action = ppo_agent.select_action(state)
            state, reward, done, _ = env.step(action)

            # saving reward and is_terminals
            ppo_agent.buffer.rewards.append(reward)
            ppo_agent.buffer.is_terminals.append(done)

            time_step +=1
            current_ep_reward += reward

            # update PPO agent
            if time_step % update_timestep == 0:
                ppo_agent.update()

            if done:
                break

        print_running_reward += current_ep_reward
        print_running_episodes += 1

        i_episode += 1

        # printing average reward
        if i_episode % print_freq_ep == 0:

            # print average reward till last episode
            print_avg_reward = print_running_reward / print_running_episodes
            print_avg_reward = round(print_avg_reward, 2)
            tracker.append([i_episode,print_avg_reward])

            eval_score = compute_avg_return(env2,ppo_agent,20)
            eval_tracker.append([i_episode,eval_score])


            print("Episode : {} \t\t Timestep : {} \t\t Average Reward : {} \t\t Eval Score: {}".format(i_episode, time_step, print_avg_reward,eval_score))
            run_ave = np.mean(np.array(tracker)[-50:,1])
            
            if max_score < run_ave and i_episode > 100:
                torch.save(ppo_agent.policy,f'{folder_name}/trial_{trial_number}_ppo_agent.pt')
                max_score = run_ave
            
            print_running_reward = 0
            print_running_episodes = 0

    env.close()
    
    np.savetxt(f'{folder_name}/trial_{trial_number}.txt', tracker, delimiter=',',fmt='%s')
    np.savetxt(f'{folder_name}/eval_trial_{trial_number}.txt', eval_tracker, delimiter=',',fmt='%s')
    torch.save(ppo_agent.policy,f'{folder_name}/trial_{trial_number}_final_ppo_agent.pt')
    
    
    return 0



In [8]:
# folder_name = 'run_1'
# train(params_array,folder_name)

In [7]:
default_params = [2e-05, 0.95, 0.001, 0.001]
best_params = [2e-02,0.95,1e-04,1e-03]

In [8]:
evaluate_and_test('best',1,best_params)

C:\Users\overl\anaconda3\envs\env1\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.mixture.gaussian_mixture module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.mixture. Anything that cannot be imported from sklearn.mixture is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\overl\anaconda3\envs\env1\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.20.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


Episode : 10 		 Timestep : 490 		 Average Reward : 21160.0 		 Eval Score: 18860.0
Episode : 20 		 Timestep : 881 		 Average Reward : 15430.0 		 Eval Score: 20775.0
Episode : 30 		 Timestep : 1307 		 Average Reward : 15730.0 		 Eval Score: 18500.0
Episode : 40 		 Timestep : 1638 		 Average Reward : 10620.0 		 Eval Score: 19540.0
Episode : 50 		 Timestep : 2030 		 Average Reward : 14990.0 		 Eval Score: 24355.0
Episode : 60 		 Timestep : 2686 		 Average Reward : 25380.0 		 Eval Score: 18925.0
Episode : 70 		 Timestep : 3057 		 Average Reward : 14910.0 		 Eval Score: 23800.0
Episode : 80 		 Timestep : 3363 		 Average Reward : 11970.0 		 Eval Score: 21390.0
Episode : 90 		 Timestep : 3976 		 Average Reward : 29400.0 		 Eval Score: 25970.0
Episode : 100 		 Timestep : 4369 		 Average Reward : 16200.0 		 Eval Score: 21190.0
Episode : 110 		 Timestep : 4915 		 Average Reward : 23660.0 		 Eval Score: 20925.0
Episode : 120 		 Timestep : 5476 		 Average Reward : 27260.0 		 Eval Score: 21865.0
Epi

Episode : 980 		 Timestep : 42460 		 Average Reward : 24940.0 		 Eval Score: 26000.0
Episode : 990 		 Timestep : 42956 		 Average Reward : 26120.0 		 Eval Score: 23450.0
Episode : 1000 		 Timestep : 43453 		 Average Reward : 25190.0 		 Eval Score: 16705.0
Episode : 1010 		 Timestep : 43729 		 Average Reward : 13140.0 		 Eval Score: 18465.0
Episode : 1020 		 Timestep : 44203 		 Average Reward : 24540.0 		 Eval Score: 20310.0
Episode : 1030 		 Timestep : 44498 		 Average Reward : 12190.0 		 Eval Score: 27680.0
Episode : 1040 		 Timestep : 44925 		 Average Reward : 20790.0 		 Eval Score: 19760.0
Episode : 1050 		 Timestep : 45410 		 Average Reward : 25490.0 		 Eval Score: 22150.0
Episode : 1060 		 Timestep : 45892 		 Average Reward : 23480.0 		 Eval Score: 20170.0
Episode : 1070 		 Timestep : 46376 		 Average Reward : 26070.0 		 Eval Score: 25350.0
Episode : 1080 		 Timestep : 46634 		 Average Reward : 12360.0 		 Eval Score: 21395.0
Episode : 1090 		 Timestep : 47117 		 Average Reward : 2

Episode : 1940 		 Timestep : 82041 		 Average Reward : 19040.0 		 Eval Score: 20510.0
Episode : 1950 		 Timestep : 82614 		 Average Reward : 30880.0 		 Eval Score: 22525.0
Episode : 1960 		 Timestep : 83028 		 Average Reward : 20240.0 		 Eval Score: 23055.0
Episode : 1970 		 Timestep : 83390 		 Average Reward : 17720.0 		 Eval Score: 20575.0
Episode : 1980 		 Timestep : 83820 		 Average Reward : 23970.0 		 Eval Score: 19705.0
Episode : 1990 		 Timestep : 84165 		 Average Reward : 17720.0 		 Eval Score: 25030.0
Episode : 2000 		 Timestep : 84619 		 Average Reward : 22870.0 		 Eval Score: 22905.0
Episode : 2010 		 Timestep : 84980 		 Average Reward : 17070.0 		 Eval Score: 27220.0
Episode : 2020 		 Timestep : 85412 		 Average Reward : 22050.0 		 Eval Score: 23940.0
Episode : 2030 		 Timestep : 85797 		 Average Reward : 19850.0 		 Eval Score: 27820.0
Episode : 2040 		 Timestep : 86178 		 Average Reward : 20230.0 		 Eval Score: 22000.0
Episode : 2050 		 Timestep : 86517 		 Average Reward :

Episode : 2890 		 Timestep : 123093 		 Average Reward : 14260.0 		 Eval Score: 17175.0
Episode : 2900 		 Timestep : 123601 		 Average Reward : 26220.0 		 Eval Score: 25930.0
Episode : 2910 		 Timestep : 123953 		 Average Reward : 17600.0 		 Eval Score: 25525.0
Episode : 2920 		 Timestep : 124386 		 Average Reward : 22450.0 		 Eval Score: 25455.0
Episode : 2930 		 Timestep : 124874 		 Average Reward : 24720.0 		 Eval Score: 30020.0
Episode : 2940 		 Timestep : 125546 		 Average Reward : 36490.0 		 Eval Score: 14925.0
Episode : 2950 		 Timestep : 125998 		 Average Reward : 22610.0 		 Eval Score: 23585.0
Episode : 2960 		 Timestep : 126389 		 Average Reward : 20090.0 		 Eval Score: 21050.0
Episode : 2970 		 Timestep : 126754 		 Average Reward : 18040.0 		 Eval Score: 23770.0
Episode : 2980 		 Timestep : 127280 		 Average Reward : 29380.0 		 Eval Score: 22665.0
Episode : 2990 		 Timestep : 127602 		 Average Reward : 17710.0 		 Eval Score: 23675.0
Episode : 3000 		 Timestep : 128215 		 Aver

...

Episode : 36640 		 Timestep : 1669614 		 Average Reward : 17990.0 		 Eval Score: 32020.0
Episode : 36650 		 Timestep : 1669899 		 Average Reward : 15300.0 		 Eval Score: 26885.0
Episode : 36660 		 Timestep : 1670476 		 Average Reward : 31980.0 		 Eval Score: 19645.0
Episode : 36670 		 Timestep : 1670943 		 Average Reward : 26650.0 		 Eval Score: 23000.0
Episode : 36680 		 Timestep : 1671361 		 Average Reward : 22740.0 		 Eval Score: 27890.0
Episode : 36690 		 Timestep : 1671813 		 Average Reward : 24350.0 		 Eval Score: 23935.0
Episode : 36700 		 Timestep : 1672242 		 Average Reward : 23920.0 		 Eval Score: 26480.0
Episode : 36710 		 Timestep : 1672675 		 Average Reward : 25870.0 		 Eval Score: 29345.0
Episode : 36720 		 Timestep : 1673093 		 Average Reward : 23300.0 		 Eval Score: 21180.0
Episode : 36730 		 Timestep : 1673634 		 Average Reward : 30770.0 		 Eval Score: 22835.0
Episode : 36740 		 Timestep : 1674116 		 Average Reward : 28360.0 		 Eval Score: 25885.0
Episode : 36750 		 Ti

Episode : 37570 		 Timestep : 1713847 		 Average Reward : 34880.0 		 Eval Score: 26795.0
Episode : 37580 		 Timestep : 1714140 		 Average Reward : 13140.0 		 Eval Score: 33125.0
Episode : 37590 		 Timestep : 1714518 		 Average Reward : 18980.0 		 Eval Score: 23290.0
Episode : 37600 		 Timestep : 1714945 		 Average Reward : 22120.0 		 Eval Score: 26840.0
Episode : 37610 		 Timestep : 1715482 		 Average Reward : 29020.0 		 Eval Score: 18240.0
Episode : 37620 		 Timestep : 1716114 		 Average Reward : 35290.0 		 Eval Score: 19550.0
Episode : 37630 		 Timestep : 1716551 		 Average Reward : 23270.0 		 Eval Score: 21995.0
Episode : 37640 		 Timestep : 1717049 		 Average Reward : 27330.0 		 Eval Score: 28740.0
Episode : 37650 		 Timestep : 1717567 		 Average Reward : 27250.0 		 Eval Score: 29165.0
Episode : 37660 		 Timestep : 1718062 		 Average Reward : 26970.0 		 Eval Score: 26195.0
Episode : 37670 		 Timestep : 1718547 		 Average Reward : 26770.0 		 Eval Score: 26630.0
Episode : 37680 		 Ti

Episode : 38500 		 Timestep : 1757241 		 Average Reward : 23360.0 		 Eval Score: 27760.0
Episode : 38510 		 Timestep : 1757754 		 Average Reward : 28530.0 		 Eval Score: 36230.0
Episode : 38520 		 Timestep : 1758298 		 Average Reward : 32210.0 		 Eval Score: 24870.0
Episode : 38530 		 Timestep : 1758840 		 Average Reward : 29740.0 		 Eval Score: 30340.0
Episode : 38540 		 Timestep : 1759302 		 Average Reward : 24210.0 		 Eval Score: 21880.0
Episode : 38550 		 Timestep : 1759816 		 Average Reward : 27860.0 		 Eval Score: 28390.0
Episode : 38560 		 Timestep : 1760162 		 Average Reward : 17520.0 		 Eval Score: 19730.0
Episode : 38570 		 Timestep : 1760506 		 Average Reward : 19240.0 		 Eval Score: 18655.0
Episode : 38580 		 Timestep : 1760831 		 Average Reward : 15640.0 		 Eval Score: 23395.0
Episode : 38590 		 Timestep : 1761086 		 Average Reward : 13260.0 		 Eval Score: 24670.0
Episode : 38600 		 Timestep : 1761518 		 Average Reward : 22360.0 		 Eval Score: 24650.0
Episode : 38610 		 Ti

Episode : 39430 		 Timestep : 1800250 		 Average Reward : 36450.0 		 Eval Score: 16915.0
Episode : 39440 		 Timestep : 1800631 		 Average Reward : 20180.0 		 Eval Score: 22775.0
Episode : 39450 		 Timestep : 1801134 		 Average Reward : 27400.0 		 Eval Score: 23810.0
Episode : 39460 		 Timestep : 1801571 		 Average Reward : 23020.0 		 Eval Score: 26590.0
Episode : 39470 		 Timestep : 1802109 		 Average Reward : 29230.0 		 Eval Score: 32975.0
Episode : 39480 		 Timestep : 1802737 		 Average Reward : 35610.0 		 Eval Score: 26950.0
Episode : 39490 		 Timestep : 1803253 		 Average Reward : 31180.0 		 Eval Score: 26235.0
Episode : 39500 		 Timestep : 1803814 		 Average Reward : 32550.0 		 Eval Score: 19405.0
Episode : 39510 		 Timestep : 1804090 		 Average Reward : 14600.0 		 Eval Score: 18515.0
Episode : 39520 		 Timestep : 1804621 		 Average Reward : 29660.0 		 Eval Score: 29540.0
Episode : 39530 		 Timestep : 1805075 		 Average Reward : 24000.0 		 Eval Score: 22685.0
Episode : 39540 		 Ti

0